# Examples of the usage

In [ ]:
# for running Ollama in Google colab
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

Once the xterm is open, we can proceed with the installation of Ollama by running the commented commands in the xterm:

In [ ]:
%xterm
# run cmd below in xterm
 # curl https://ollama.ai/install.sh | sh
 # ollama serve

In [ ]:
%pip install urban-worm --upgrade

Import dataset class

In [ ]:
from urbanworm import UrbanDataSet

In [6]:
bbox = (-83.235572,42.348092,-83.235154,42.348806)
data = UrbanDataSet()
data.bbox2Buildings(bbox)

'5 buildings found in the bounding box.'

In [5]:
data.units

geometry
0  POLYGON ((-83.23534 42.3488, -83.23523 42.3488...
1  POLYGON ((-83.23533 42.34865, -83.23522 42.348...
2  POLYGON ((-83.23533 42.3485, -83.23522 42.3485...
3  POLYGON ((-83.23533 42.34835, -83.23521 42.348...
4  POLYGON ((-83.23532 42.34821, -83.23521 42.348...

In [7]:
data.units.plot()

<Axes: >

In [8]:
system = '''
    Given a top view image, you are going to roughly estimate house conditions. Your answer should be based only on your observation. 
    The format of your response must include question, answer (yes or no), explaination (within 50 words)
'''
prompt = {
    'top':'''
        Is there any damage on the roof?
    '''
}

# inspect the aerial images only (with type='top')
res = data.loopUnitChat(system=system, prompt=prompt, type='top', epsg=2253)

The output can be assigned to a new variable. It is also automatically stored in the data structure and can be called.

In [10]:
data.to_gdf()

geometry                top_view_question1  \
0  POINT (-83.23528 42.34875)  Is there any damage on the roof?   
1   POINT (-83.23528 42.3486)  Is there any damage on the roof?   
2  POINT (-83.23527 42.34845)  Is there any damage on the roof?   
3  POINT (-83.23527 42.34831)  Is there any damage on the roof?   
4  POINT (-83.23526 42.34816)  Is there any damage on the roof?   

  top_view_answer1                              top_view_explanation1  \
0               no  The image does not show any visible signs of d...   
1               no  The image does not show any visible signs of d...   
2               no  The image does not show any visible signs of d...   
3               no  The image does not show any visible signs of d...   
4               no  The image is too blurry to determine if there ...   

                                     top_view_base64  
0  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...  
1  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...  
2  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...  
3  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...  
4  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...

With `type = 'both'` (The default is `top`.), the method `loopUnitChat` can inspect both aerial and street view images. 
With `multi = True` (The default is `False`), for each unit, three closest street views will be collected for inference. This will take longer time to process the data.

In this case, the instruction and prompt will be little bit different and more detailed for specifying questions for each type of image.

In [ ]:
system = '''
    Given a top view image or street view images, you are going to roughly estimate house conditions. 
    Your answer should be based only on your observation. 
    The format of your response must include question, answer (yes or no), explaination (within 50 words) for each question.
'''

prompt = {
    'top': '''
        Is there any damage on the roof?
    ''',
    'street': '''
        Is the house occupied?
        Is any window broken or boarded?
        Is the door missing or boarded?
        Is the wall missing or damaged?
        Is the yard maintained well?
    '''
}

# add the Mapillary key
data.mapillary_key = 'MLY|...'
# inspect both the aerial and street view images (with type='both')
res = data.loopUnitChat(system=system, prompt=prompt, type='both', epsg=2253, multi=True)

In [12]:
data.to_gdf()

geometry                top_view_question1  \
0  POINT (-83.23528 42.34875)  Is there any damage on the roof?   
1   POINT (-83.23528 42.3486)  Is there any damage on the roof?   
2  POINT (-83.23527 42.34845)  Is there any damage on the roof?   
3  POINT (-83.23527 42.34831)  Is there any damage on the roof?   
4  POINT (-83.23526 42.34816)  Is there any damage on the roof?   

  top_view_answer1                              top_view_explanation1  \
0               no  The roof appears to be intact and undamaged, w...   
1               no  The roof appears to be intact, with no visible...   
2               no  The roof appears to be intact, with no visible...   
3               no  The roof appears to be intact, with no visible...   
4               no  The roof appears to be intact, with no visible...   

                               street_view_question1 street_view_answer1  \
0  [Is the house occupied?, Is the house occupied...      [yes, No, yes]   
1  [Is the house occupied?, Is the house occupied...      [yes, No, Yes]   
2  [Is the house occupied?, Is the house occupied...        [No, No, No]   
3  [Is the house occupied?, Is the house occupied...        [No, No, No]   
4  [Is the house occupied?, Is the house occupied...       [No, yes, No]   

                            street_view_explanation1  \
0  [There are lights on in some of the windows., ...   
1  [There are objects in front of the house, sugg...   
2  [There are no cars parked in front of the hous...   
3  [There is no car parked in front of the house,...   
4  [There are no cars parked in front of the hous...   

                               street_view_question2 street_view_answer2  \
0  [Is any window broken or boarded?, Is any wind...       [no, Yes, no]   
1  [Is any window broken or boarded?, Is any wind...       [no, Yes, No]   
2  [Is any window broken or boarded?, Is any wind...     [Yes, Yes, Yes]   
3  [Is any window broken or boarded?, Is any wind...     [Yes, Yes, Yes]   
4  [Is any window broken or boarded?, Is any wind...      [Yes, no, Yes]   

                            street_view_explanation2  ... street_view_answer3  \
0  [All the windows appear to be intact, One of t...  ...        [no, No, no]   
1  [All windows appear to be intact and unobstruc...  ...        [no, No, No]   
2  [One of the windows on the second floor appear...  ...        [No, No, No]   
3  [One of the windows on the second floor appear...  ...        [No, No, No]   
4  [One of the windows on the second floor appear...  ...        [No, no, No]   

                            street_view_explanation3  \
0  [The front door is visible and appears to be i...   
1  [The front door is visible and appears to be i...   
2  [The front door is visible and does not appear...   
3  [The front door is visible and does not appear...   
4  [The front door is visible and does not appear...   

                               street_view_question4 street_view_answer4  \
0  [Is the wall missing or damaged?, Is the wall ...       [no, Yes, no]   
1  [Is the wall missing or damaged?, Is the wall ...       [no, Yes, No]   
2  [Is the wall missing or damaged?, Is the wall ...     [Yes, Yes, Yes]   
3  [Is the wall missing or damaged?, Is the wall ...     [Yes, Yes, Yes]   
4  [Is the wall missing or damaged?, Is the wall ...      [Yes, no, Yes]   

                            street_view_explanation4  \
0  [All walls appear to be intact and undamaged, ...   
1  [All walls appear to be intact and undamaged.,...   
2  [There appears to be some damage or disrepair ...   
3  [There appears to be some damage or disrepair ...   
4  [There appears to be some damage or disrepair ...   

                               street_view_question5 street_view_answer5  \
0  [Is the yard maintained well?, Is the yard mai...      [yes, No, yes]   
1  [Is the yard maintained well?, Is the yard mai...      [yes, No, Yes]   
2  [Is the yard maintained well?, Is the yard mai...        [No, No, No]   
3  [Is